# Week 7 — Part 04: Testing strategy (pytest vs smoke tests)

**Estimated time:** 45–75 minutes

## What success looks like (end of Part 04)

- You can name at least 3 checks (happy path, edge case, failure case).
- Your checks verify artifacts/JSON validity rather than exact LLM text.
- You can run a minimal smoke test that creates expected files under `output/`.

### Checkpoint

After running this notebook, you should be able to:

- point to one “happy path” check
- point to one “expected failure” check

## Learning Objectives

- Distinguish unit tests from smoke tests
- Write a minimal test plan (happy, edge, failure cases)
- Check artifacts and JSON validity instead of exact text
- Choose pytest or a smoke-test script for Level 1

## Overview

Tests are executable checks that protect you from regressions.

---

## Underlying theory: tests reduce uncertainty when you change things

Every change introduces risk. Tests keep the system stable while you iterate.

Two common layers:

- **unit tests**: small, fast checks (parsing, validation, file handling)
- **smoke tests**: end-to-end checks (pipeline runs and produces artifacts)

For LLM projects, you often cannot assert exact text outputs. Instead assert:

- output is valid JSON
- required keys exist
- file artifacts are created
- failures are handled gracefully

For Level 1 you can choose:

- `pytest` unit tests (preferred)
- or a `smoke_test.py` + manual checklist (acceptable)

## Minimal test plan (3+ cases)

You should have at least:

- **happy path**: normal input works
- **edge case**: missing values or tiny CSV
- **failure case**: missing file / invalid schema

Practical tip: smoke tests can be “one command” checks that run in CI later. The key is making them deterministic enough to be repeatable.

import json
import tempfile
from pathlib import Path


def require_file(path: str) -> Path:
    p = Path(path).expanduser()
    if not p.exists():
        raise FileNotFoundError("Input file not found: %s" % p)
    if p.stat().st_size == 0:
        raise ValueError("Input file is empty: %s" % p)
    return p


def test_require_file_missing(tmp_path):
    # TODO: implement pytest unit test for missing file.
    # Example: call require_file() and assert FileNotFoundError.
    missing = Path(tmp_path) / "missing.csv"
    try:
        _ = require_file(str(missing))
    except FileNotFoundError:
        return
    raise AssertionError("expected FileNotFoundError")


def test_require_file_empty(tmp_path):
    # TODO: implement pytest unit test for empty file.
    p = Path(tmp_path) / "empty.csv"
    p.write_text("", encoding="utf-8")
    try:
        _ = require_file(str(p))
    except ValueError:
        return
    raise AssertionError("expected ValueError")


def smoke_test_pipeline() -> None:
    # TODO: run an end-to-end pipeline with a tiny input.
    # Verify artifacts exist: output/report.json, output/report.md
    out_dir = Path("output")
    out_dir.mkdir(exist_ok=True)
    (out_dir / "report.json").write_text(json.dumps({"ok": True}, indent=2), encoding="utf-8")
    (out_dir / "report.md").write_text("# Report\n\nOK", encoding="utf-8")


with tempfile.TemporaryDirectory() as d:
    test_require_file_missing(d)
    test_require_file_empty(d)

smoke_test_pipeline()
print("wrote:", Path("output") / "report.json")
print("Implement pytest tests + smoke_test_pipeline().")

## References

- pytest docs: https://docs.pytest.org/

## Appendix: Solutions (peek only after trying)

Reference implementations for pytest-style tests and a smoke test.

In [ ]:
def test_require_file_missing(tmp_path):
    import pytest

    missing = Path(tmp_path) / "missing.csv"
    with pytest.raises(FileNotFoundError):
        _ = require_file(str(missing))


def test_require_file_empty(tmp_path):
    import pytest

    p = Path(tmp_path) / "empty.csv"
    p.write_text("", encoding="utf-8")
    with pytest.raises(ValueError):
        _ = require_file(str(p))


def smoke_test_pipeline() -> None:
    out_dir = Path("output")
    out_dir.mkdir(exist_ok=True)
    (out_dir / "report.json").write_text(json.dumps({"ok": True}, indent=2), encoding="utf-8")
    (out_dir / "report.md").write_text("# Report\n\nOK", encoding="utf-8")


print("solution smoke wrote:", Path("output") / "report.json")